# 02_entiry-extract
- entity extraction from news description using trasnformer `Davlan/xlm-roberta-base-ner-hrl`

### install extra library

In [1]:
!pip install transformers torch
!pip install sentencepiece

### read stored data from DataPlatform

In [2]:
from dsmlibrary.datanode import DataNode
import json
from tqdm.auto import tqdm
import pandas as pd

In [3]:
dir_raw_id = 96
dir_process_id = 97

In [4]:
datanode = DataNode()

Init DataNode sucessful!


In [5]:
meta, fp = datanode.get_file(datanode.get_file_id(directory_id=dir_raw_id, name='Tops_raw.json'))
meta

{'owner': 'pranodnard',
 'human_size': '81.32 KB',
 'name': 'Tops_raw.json',
 'description': '-',
 'directory': 96,
 'path': 'pranodnard/DSI310-25/raw/Tops_raw.json'}

In [ ]:
data = json.load(fp)
data

In [7]:
df1 = pd.Series(data)
df1

0      {'Name': 'ซันบลาสน้ำแอปเปิ้ลเชอร์รี่ออร์แกนิก1...
1      {'Name': 'ซันบลาสน้ำแอปเปิ้ลผสมมะม่วงออร์แกนิก...
2      {'Name': 'แพร์คอเรลลา', 'Price': '299', 'Unit'...
3      {'Name': 'เบบี้เนเชอร่าข้าวไรซ์เบอร์รี่ออร์แกน...
4      {'Name': 'โดโซะเบบี้ไบท์ข้าวหอมญี่ปุ่นอบกรอบออ...
                             ...                        
228    {'Name': 'โทฟุซังนมถั่วเหลืองออร์แกนิคสูตรหวาน...
229    {'Name': 'โทฟุซังนมถั่วเหลืองออร์แกนิครสออริจิ...
230    {'Name': 'เพียวฮาร์เวสต์ออสซี่ดรีมน้ำนมข้าวกล้...
231    {'Name': 'บอนซอยนมถั่วเหลืองออร์แกนิคขนาด 1ลิต...
232    {'Name': 'ดีน่าโทนิวน้ำเต้าหู้ออร์แกนิค 180มล....
Length: 233, dtype: object

### NER (NameEntitiesRegcognition)
- extract entity from news description

In [8]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

In [9]:
tokenizer = AutoTokenizer.from_pretrained("Davlan/xlm-roberta-base-ner-hrl")
model = AutoModelForTokenClassification.from_pretrained("Davlan/xlm-roberta-base-ner-hrl")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

In [10]:
example = "My name is Wolfgang and I live in Berlin"

ner_results = nlp(example)
print(ner_results)

[{'entity': 'B-PER', 'score': 0.9993357, 'index': 4, 'word': '▁Wolfgang', 'start': 10, 'end': 19}, {'entity': 'B-LOC', 'score': 0.9990395, 'index': 9, 'word': '▁Berlin', 'start': 33, 'end': 40}]


In [11]:
import re

In [12]:
def post_process(preds):
    data = []
    for i in range(len(preds)): 
        a = preds[i]
        text = a.get('word')
        for j in range(i, len(preds)):
            b = preds[j]
            if b.get('start') == a.get('end'):
                text += b.get('word')
                a = b
        data.append({
            'type': a.get('entity'),
            'text': text,
            'Product_Name': Tops_raw.get('Name'),
            'Product_Price': Tops_raw.get('Price'),
            'Product_Unit': Tops_raw.get('Unit'),
            'Product_Category': Tops_raw.get('Category'),
        })
    tmp = []
    for elm in data:
        text = elm.get('text')
        if bool(re.match("[ก-๛]|▁", text)):
            elm.update({'text': text.replace('▁', ' ')})
            tmp.append(elm)
    return tmp

In [13]:
text = data[0]['Name']
text

'ซันบลาสน้ำแอปเปิ้ลเชอร์รี่ออร์แกนิก100เปอร์เซ็นต์ 200มล.'

In [14]:
output = nlp(text)
output

[{'entity': 'B-ORG',
  'score': 0.93881553,
  'index': 2,
  'word': 'ซัน',
  'start': 0,
  'end': 3},
 {'entity': 'I-ORG',
  'score': 0.7931354,
  'index': 3,
  'word': 'บ',
  'start': 3,
  'end': 4},
 {'entity': 'I-ORG',
  'score': 0.80668145,
  'index': 4,
  'word': 'ลาส',
  'start': 4,
  'end': 7}]

In [15]:
post_process(output)

NameError: name 'Tops_raw' is not defined

### Process news to spo

In [16]:
spo = []
for Tops_raw in tqdm(data):
    Name = Tops_raw.get('Name') 
    for kw in Name:
        spo.append({
            'Product_Name': Tops_raw.get('Name', ""),
            'Product_Price': Tops_raw.get('Price', ""),
            'Product_Unit': Tops_raw.get('Unit', ""),
            'Product_Category': Tops_raw.get('Category', ""),
        })

  0%|          | 0/233 [00:00<?, ?it/s]

In [17]:
df = pd.DataFrame(spo)
df['Product_Price'] = df['Product_Category'].apply(lambda x: x[:2])
df

,Product_Name,Product_Price,Product_Unit,Product_Category
0,ซันบลาสน้ำแอปเปิ้ลเชอร์รี่ออร์แกนิก100เปอร์เซ็...,10,ชิ้น,100%Juice
1,ซันบลาสน้ำแอปเปิ้ลเชอร์รี่ออร์แกนิก100เปอร์เซ็...,10,ชิ้น,100%Juice
2,ซันบลาสน้ำแอปเปิ้ลเชอร์รี่ออร์แกนิก100เปอร์เซ็...,10,ชิ้น,100%Juice
3,ซันบลาสน้ำแอปเปิ้ลเชอร์รี่ออร์แกนิก100เปอร์เซ็...,10,ชิ้น,100%Juice
4,ซันบลาสน้ำแอปเปิ้ลเชอร์รี่ออร์แกนิก100เปอร์เซ็...,10,ชิ้น,100%Juice
...,...,...,...,...
9809,ดีน่าโทนิวน้ำเต้าหู้ออร์แกนิค 180มล.แพค4,UH,แพค,UHTDairyFree
9810,ดีน่าโทนิวน้ำเต้าหู้ออร์แกนิค 180มล.แพค4,UH,แพค,UHTDairyFree
9811,ดีน่าโทนิวน้ำเต้าหู้ออร์แกนิค 180มล.แพค4,UH,แพค,UHTDairyFree
9812,ดีน่าโทนิวน้ำเต้าหู้ออร์แกนิค 180มล.แพค4,UH,แพค,UHTDairyFree


### write DataNode to DataPlatform

In [18]:
datanode.write(df=df, directory=dir_process_id, name="spo", profiling=True, lineage=[datanode.get_file_id(directory_id=dir_raw_id, name='Tops_raw.json')])

{'sucess': True,
 'file_id': 175,
 'path': 'pranodnard/DSI310-25/process/spo.parquet',
 'lineage': True}